In [1]:
from google.colab import drive
drive.mount('/content/drive')

!cp -R "/content/drive/Shareddrives/Data/Deploy Land Use/static" "/content/"
!cp -R "/content/drive/Shareddrives/Data/Deploy Land Use/templates" "/content/"
!mkdir upload

Mounted at /content/drive


In [2]:
import os, re
import tensorflow as tf

_path = "/content/drive/Shareddrives/Data/Model Land Use"
models = os.listdir(_path)
best_acc = 0
best_model = ""

regex = r"(\d+)(?:\.(\d{1,}))"
for _model in models:
  acc = re.search(regex,_model)
  cur_acc = float(acc.group(0))
  if cur_acc > best_acc:
    best_acc = cur_acc
    best_model = _model

model= tf.keras.models.load_model(os.path.join(_path, best_model))
print(f"Model used: {best_model}")

Model used: model_MobileNet Acc:0.9541666507720947.h5


In [3]:
!pip install flask-ngrok

In [4]:
class_label = {0: 'agricultural',
              1: 'airplane',
              2: 'baseballdiamond',
              3: 'beach',
              4: 'buildings',
              5: 'chaparral',
              6: 'denseresidential',
              7: 'forest',
              8: 'freeway',
              9: 'golfcourse',
              10: 'harbor',
              11: 'intersection',
              12: 'mediumresidential',
              13: 'mobilehomepark',
              14: 'overpass',
              15: 'parkinglot',
              16: 'river',
              17: 'runway',
              18: 'sparseresidential',
              19: 'storagetanks',
              20: 'tenniscourt'}

In [5]:
from __future__ import division, print_function
# coding=utf-8
import sys
import os
import glob
import re
import numpy as np

# Keras
from keras.models import load_model
from keras.preprocessing import image

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from flask_ngrok import run_with_ngrok


import cv2
import numpy as np
# Define a flask app
app = Flask(__name__)
run_with_ngrok(app)

def model_predict(img_path, model):
    img = cv2.imread(img_path)

    img = cv2.resize(img,(150,150))
    img = np.reshape(img,[1,150,150,3])
    img_arr = np.zeros((1,150,150,3))
    img_arr[0, :, :, :] = img / 255.

    y_pred = model.predict(img_arr)
    return class_label[np.argmax(y_pred)]


@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        # Save the file to ./uploads
        basepath = os.path.dirname("/content/upload")
        file_path = os.path.join(
            basepath, 'upload', secure_filename(f.filename))
        f.save(file_path)

        # Make prediction
        preds = model_predict(file_path, model)

        result = str(preds)
        return result
    return None


if __name__ == '__main__':
    app.run()



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e859-35-231-105-101.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [22/Sep/2021 04:19:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 04:19:41] "GET /static/js/main.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 04:19:41] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 04:19:43] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Sep/2021 04:19:58] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 04:20:18] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 04:21:05] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 04:22:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 04:22:17] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 04:22:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 04:23:04] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 04:24:08] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 04:24:46] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 04:24:50] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Sep/2021 04:25:28] "POST /predi